In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [51]:
from sklearn import *
import sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [52]:
data_path = '/home/genughaben/datasets/kaggle/titanic'

In [53]:
!ls {data_path}

gender_submission.csv  test.csv  train.csv


In [57]:
train_data_path = f'{data_path}/train.csv'
test_data_path = f'{data_path}/test.csv'
test_submission_path = f'{data_path}/gender_submission.csv'
sumission_path = f'{data_path}/submission.csv'

In [73]:
# column names
df = pd.read_csv(test_data_path)
column_names = list(df)
row_count = len(df)
print("column names: \n",column_names)
print('row count: ', row_count)

column names: 
 ['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
row count:  418


In [91]:
def create_submission(test_data, field_name, predictions=None):
    passengers = test_data[[field_name]]
    if predictions == None:
        # if no predictions provided all predictions are set to zero
        predictions = [0] * row_count
    predictions_series = pd.Series(predictions, index=passengers.index)
    passengers.loc[:, 'Survived'] = predictions_series
    return passengers
    
submission_df = create_submission(test_data, "PassengerId")
submission_df.head()

<class 'list'>


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [146]:
train_data = pd.read_csv(train_data_path)
test_data =  pd.read_csv(test_data_path)
test_submission =  pd.read_csv(test_submission_path)

In [147]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [148]:
train_data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [159]:
y = train_data['Survived']
X_numeric = train_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin']]

In [160]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

In [161]:
sex_cat = train_data['Sex']
sex_cat_1hot = label_encoder.fit_transform(sex_cat)

In [162]:
from sklearn.preprocessing import LabelBinarizer
binarizer_encoder = LabelBinarizer()

In [163]:
cabin_cat = train_data['Cabin']
cabin_cat = cabin.fillna('Z')
cabin_cat_1char = [s[0] for s in list(cabin)]
cabin_cat_1char_1hot = binarizer_encoder.fit_transform(cabin_cat_1char)

In [175]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='constant', fill_value='Z')
cabin_cat = pd.DataFrame(train_data['Cabin'])
cabin_cat_imp = imputer.fit_transform(cabin_cat)
type(cabin_cat_imp)

numpy.ndarray

In [185]:
from sklearn.base import BaseEstimator, TransformerMixin

class FirstCharacterExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return [s[0] for s in list(X.flatten())]
    
char_extractor = FirstCharacterExtractor()
cabin_first_char = char_extractor.transform(cabin_cat_imp)

In [164]:
binarizer_encoder = LabelBinarizer()
embarked_cat = train_data['Embarked']
embarked_cat.head()
#embarked_cat.value_counts()
embarked_cat_1hot = binarizer_encoder.fit_transform(embarked_cat.astype(str))

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [110]:
# X.loc[:, 'Cabin_lvl'] = pd.Series(predictions, index=passengers.index)
# X['Cabin'].value_counts()


array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])